# Joseph Development 7th Nov


In [1]:

from transformer_lens.HookedTransformer import HookedTransformer
from transformer_lens.loading_from_pretrained import OFFICIAL_MODEL_NAMES

OFFICIAL_MODEL_NAMES[1:10]
model = HookedTransformer.from_pretrained("EleutherAI/gpt-j-6B")

from src.probe_training_runner

/opt/conda/envs/alphabetical_probe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['gpt2-medium',
 'gpt2-large',
 'gpt2-xl',
 'distilgpt2',
 'facebook/opt-125m',
 'facebook/opt-1.3b',
 'facebook/opt-2.7b',
 'facebook/opt-6.7b',
 'facebook/opt-13b']

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/gpt-j-6B into HookedTransformer


In [3]:
import nltk
from nltk.corpus import words

nltk.download("words", quiet=True)


def get_valid_word_idxs(list_of_strings):
    english_words = set(w.lower() for w in words.words())
    valid_english_word_idxs = []

    for i, s in enumerate(list_of_strings):
        # Remove leading "Ġ" character if present
        cleaned_string = s.lstrip("Ġ")
        # Strip any other leading/trailing whitespace and convert to lowercase
        cleaned_string = cleaned_string.strip().lower()
        # Check if the word is in the set of English words
        if cleaned_string in english_words:
            valid_english_word_idxs.append(i)

    return valid_english_word_idxs


import numpy as np

token_strings = list(model.tokenizer.get_vocab().keys())
valid_words = get_valid_word_idxs(token_strings)
print(len(valid_words))
# randomly sample 100 words from the valid words
for i in np.random.choice(valid_words, 10):
    print(token_strings[i])

25950
First
Ġvascular
ĠBan
ĠBadge
Ġhumane
Ġmissile
Ġapparently
Ġmembrane
Ġrenal
Ġposition


# Get Training Data

In [4]:
tokenizer = model.tokenizer
vocab = tokenizer.get_vocab()
# filter vocab to only include valid words
vocab = {k: v for k, v in vocab.items() if v in valid_words}
len(vocab)

25950

In [143]:
make_weights_for_balanced_classes(torch.tensor([True,True,False,False,False,False,False,False,False,False]))

[tensor(5.),
 tensor(5.),
 tensor(1.2500),
 tensor(1.2500),
 tensor(1.2500),
 tensor(1.2500),
 tensor(1.2500),
 tensor(1.2500),
 tensor(1.2500),
 tensor(1.2500)]

In [150]:
import re
import torch 
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    WeightedRandomSampler,
    random_split,

)

def get_classification_indices(vocab, regex_pattern):
    # get indices of words that match the regex pattern
    indices = [
        v for k, v in vocab.items() if re.match(regex_pattern, k.strip().strip("Ġ"))
    ]
    # get the indices of the words that don't match the regex pattern
    not_indices = [
        v for k, v in vocab.items() if not re.match(regex_pattern, k.strip().strip("Ġ"))
    ]
    return indices, not_indices


def get_regex_pattern(letter, criterion):
    # Escaping the letter in case it has a special meaning in regex (e.g., '.' or '*')
    escaped_letter = re.escape(letter)

    if criterion == "contains_any":
        # Match the letter anywhere in the string
        return rf"{escaped_letter}"
    elif criterion == "starts":
        # Match strings that start with the letter
        return rf"^{escaped_letter}"
    elif criterion == "ends":
        # Match strings that end with the letter
        return rf"{escaped_letter}$"
    elif criterion == "contains_1":
        # Match strings that contain the letter exactly once
        return rf"^[^{escaped_letter}]*{escaped_letter}[^{escaped_letter}]*$"
    else:
        raise ValueError(f"Unknown criterion: {criterion}")


def make_weights_for_balanced_classes(labels):
    # Count of instances in each class
    n_total = len(labels)
    n_pos = sum(labels)
    n_neg = len(labels) - n_pos


    # Weight for each sample
    weight_per_class = {True: n_total / n_pos, False: n_total / n_neg}

    # Assign weight to each sample
    weights = [weight_per_class[label.item()] for label in labels]
    
    return weights

def filter_vocab(vocab):
    english_words = set(w.lower() for w in words.words())
    new_vocab = {}
    for word, index in vocab.items():
        clean_word = word.lstrip("Ġ").strip().lower()
        if clean_word in english_words and bool(roman_char_regex.match(clean_word)):
            new_vocab[word] = index
    return new_vocab


def get_letter_dataset(
    criterion,
    target,
    embeddings,
    vocab,
    batch_size=32,
    rebalance=False,
    test_proportion=0.2,
):
    
    
    new_vocab = filter_vocab(vocab)
    vocab_tokens = list(new_vocab.keys())
    new_embeddings = embeddings[list(new_vocab.values())]
    # original_indices = list(new_vocab.values())
    # index_mapping = {v: i for i, (k, v) in enumerate(new_vocab.items())}

    # get indices of words that start with the letter A
    regex_pattern = get_regex_pattern(target, criterion)
    labels = [bool(re.search(regex_pattern, k.strip().strip("Ġ"))) for k in vocab_tokens]
    labels = torch.tensor(labels, dtype=torch.bool, device=new_embeddings.device, requires_grad=False)
    
    dataset = TensorDataset(new_embeddings, labels)
    
    train_size = int((1 - test_proportion) * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    # rebalance
    if rebalance:
        train_labels = dataset.tensors[1][train_dataset.indices]
        train_weights = make_weights_for_balanced_classes(train_labels)
        sampler = WeightedRandomSampler(train_weights, len(train_weights))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
        )
    else:
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
        )

    return train_loader, test_loader


train_loader, test_loader = get_letter_dataset(
    criterion="starts",
    target="a",
    embeddings=model.W_E,
    vocab=vocab,
    batch_size=128,
    rebalance=True,
)

print(len(train_loader))
print(len(test_loader))
# give me a batch
for batch in train_loader:
    print(batch[0].shape)
    print(batch[1])
    # get proportions of positive and negative labels
    print(sum(batch[1]) / len(batch[1]))
    break

163
41
torch.Size([128, 4096])
tensor([ True,  True,  True,  True,  True, False, False, False, False,  True,
        False, False, False,  True, False, False, False,  True,  True,  True,
         True, False,  True, False,  True, False,  True, False, False, False,
         True,  True,  True, False,  True,  True,  True,  True, False, False,
        False, False,  True, False,  True,  True, False, False,  True, False,
        False,  True, False,  True, False,  True, False,  True, False, False,
        False,  True,  True, False,  True, False,  True,  True, False,  True,
         True, False, False, False, False, False,  True,  True, False,  True,
         True, False, False, False, False, False, False,  True, False,  True,
        False, False,  True, False, False, False,  True, False, False,  True,
        False,  True,  True, False,  True,  True,  True,  True,  True, False,
        False, False, False, False, False, False, False, False,  True,  True,
        False, False,  True, Fals

In [117]:
train_dataset

# Basic Training Script

# Actual training Code

Runner needs, model, config args, probe type

In [153]:
# import auto tqdm
import wandb
import torch
import torchmetrics
import torch.optim as optim
from tqdm.auto import tqdm
import torch.nn as nn
from src.probes import LinearProbe, MLPProbe
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
)

def all_probe_training_runner(
    # data
    embeddings,
    vocab,
    # task
    alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ",  # Task
    criteria_mode="anywhere",  # "anywhere", "starting" or "posN" (where N is a digit)
    # arch
    probe_type="linear",
    # hyperparameters
    num_epochs=100,  # Define number of training epochs:
    batch_size=32,
    learning_rate=0.001,
    train_test_split=0.8,
    rebalance=False,
    # other config
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    use_wandb=False,
):
    group_name = wandb.util.generate_id() + "_" + alphabet

    # Initialize an empty dictionary to store the learned weights for all letters (or, equivalently, 26 "directions", one for each linear probe)
    embeddings_dim = embeddings.shape[1]
    probes = {letter: LinearProbe(embeddings_dim).to(device) for letter in alphabet}

    # Now loop over the alphabet and train/validate a probe for each letter:

    for i, letter in enumerate(alphabet):
        model = probes[letter]
        nn.init.xavier_uniform_(model.fc.weight)  # best way to do this?

        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.1, patience=10, verbose=True
        )

        if use_wandb:
            config = {
                "letter": letter,
                "criteria_mode": criteria_mode,
                "model_name": "gpt-j",
                "probe_type": probe_type,
                "train_test_split": train_test_split,
                "rebalance": rebalance,
                "batch_size": batch_size,
                "learning_rate": learning_rate,
                "num_epochs": num_epochs,
                "device": device,
            }

            wandb.init(
                project="letter_presence_probes",
                group=group_name,
                config=config,
            )

        train_loader, test_loader = get_letter_dataset(
            criterion=criteria_mode,
            target=letter,
            embeddings=embeddings,
            vocab=vocab,
            batch_size=batch_size,
            rebalance=rebalance,
            test_proportion=1 - train_test_split,
        )

        # Train the probe for the current letter:
        model, other_artifacts = train_letter_probe_runner(
            model=probes[letter],
            train_loader=train_loader,
            test_loader=test_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=num_epochs,
            device=device,
            use_wandb=use_wandb,
        )
        
        probes[letter] = model

    if use_wandb:
        # Start a new W&B run for logging the aggregate artifact
        wandb.init(project="letter_presence_probes", name="aggregate_artifact_logging")
        # create_and_log_artifact(
        #     probes,
        #     "all_probe_weights",
        #     "model_tensors",
        #     "All case-insensitive letter presence probe weights tensor",
        # )
        
        # log all probe weights
        artifacts = []
        for letter, probe in probes.items():
            artifact = wandb.Artifact(
                letter,
                type="model_tensors",
                description=f"Letter presence probe weights tensor for letter {letter}",
            )
            artifact.add_file(f"probe_{letter}_{criteria_mode}.pt", probe.fc.weight.detach())
            artifacts.append(artifact)
        
        wandb.log_artifact(artifacts)
        
        wandb.finish()  # End the run

    return probes


def train_letter_probe_runner(
    model,
    train_loader,
    test_loader,
    num_epochs,
    optimizer: optim.Optimizer,
    scheduler: optim.lr_scheduler.ReduceLROnPlateau,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    use_wandb=False,
):
    loss_fn = nn.BCEWithLogitsLoss()

    steps = 0
    metric_frequency = 10
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode

        pbar = tqdm(train_loader)

        for batch_embeddings, batch_labels in pbar:
            # Move your data to the chosen device during the training loop and ensure they're float32
            # By explicitly converting to float32, you ensure that the data being fed into your model has the expected data type, and this should resolve the error you en
            # batch_embeddings = batch_embeddings
            # batch_labels = batch_labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(batch_embeddings).squeeze()
            loss = loss_fn(outputs, batch_labels.to(device).float())
            loss.backward()
            optimizer.step()

            if steps % metric_frequency == 0:
                with torch.no_grad():
                    probs = torch.sigmoid(outputs).detach() > 0.5
                    
                    accuracy = torchmetrics.functional.accuracy(probs, batch_labels, task = 'binary', num_classes=2).item()
                    precision = torchmetrics.functional.precision( probs, batch_labels, task= 'binary', num_classes=2).item()
                    recall = torchmetrics.functional.recall(probs, batch_labels, 'binary', num_classes=2).item()
                    f1 = torchmetrics.functional.f1_score(probs, batch_labels, 'binary', num_classes=2).item()
        
                    # predictions
                    pbar.set_description(
                        f"Epoch {epoch+1} / {num_epochs} | Loss: {loss:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}"
                    )
                    if use_wandb:
                        # Log metrics to W&B
                        wandb.log(
                            {
                                "training_metrics/precision": precision,
                                "training_metrics/recall": recall,
                                "training_metrics/loss": loss.item(),
                                # "training_metrics/f1_score": f1_score,
                                "metrics/learning_rate": optimizer.param_groups[0]["lr"],
                            },
                            step=steps,
                        )

            steps += 1
            
        # EVALUATION (VALIDATION) PHASE

        # Set the model to evaluation mode
        model.eval()

        # Keep track of correct predictions and total predictions
        correct_preds = 0
        total_preds = 0
        validation_loss = 0.0

        with torch.no_grad():  # Ensure no gradients are computed during validation
            all_labels = []  # Store all true labels
            all_predictions = []  # Store all model predictions

            for batch_embeddings, batch_labels in test_loader:
                batch_embeddings = batch_embeddings.to(
                    device
                ).float()  # Ensure embeddings are on the correct device and dtype
                batch_labels = batch_labels.to(
                    device
                ).float()  # Ensure labels are on the correct device and dtype

                outputs = model(batch_embeddings).squeeze()

                # Calculate loss on validation data
                loss = loss_fn(outputs, batch_labels)
                validation_loss += loss.item()  # Update validation loss

                # Convert outputs to probabilities
                probs = torch.sigmoid(outputs)
                predictions = (probs > 0.5).float()

                # Update correct and total predictions
                correct_preds += (predictions == batch_labels).sum().item()
                total_preds += batch_labels.size(0)

                # Append batch labels and predictions to all_labels and all_predictions
                all_labels.append(batch_labels.cpu().numpy())
                all_predictions.append(predictions.cpu().numpy())

            # Flatten all_labels and all_predictions lists and convert to numpy arrays
            all_labels = np.concatenate(all_labels)
            all_predictions = np.concatenate(all_predictions)

            # Compute F1 Score
            f1 = f1_score(all_labels, all_predictions)

            validation_loss /= len(test_loader)  # Get the average validation loss

            # Calculate accuracy and average loss
            accuracy = correct_preds / total_preds
            precision = precision_score(all_labels, all_predictions)
            recall = recall_score(all_labels, all_predictions)

            # print(f"eval_metrics/accuracy: {accuracy * 100:.2f}%")
            # print(f"eval_metrics/precision: {precision * 100:.2f}%")
            # print(f"eval_metrics/recall: {recall * 100:.2f}%")
            # print(f"eval_metrics/loss: {validation_loss:.4f}")
            # print(f"eval_metrics/f1: {f1:.4f}\n")
            scheduler.step(validation_loss)

            if use_wandb:
                wandb.log(
                    {
                        "eval_metrics/accuracy": accuracy * 100,
                        "eval_metrics/precision": precision * 100,
                        "eval_metrics/recall": recall * 100,
                        "eval_metrics/loss": validation_loss,
                        "eval_metrics/f1": f1,
                    }
                )

    if use_wandb:
        wandb.finish()

    return model, 0

probe_weights_tensor = all_probe_training_runner(
    embeddings=model.W_E.detach(),
    vocab=vocab,
    # alphabet="A",
    criteria_mode="starts",
    probe_type="linear",
    num_epochs=30,
    batch_size=32,
    learning_rate=0.005,
    train_test_split=0.95,
    rebalance=True,
    use_wandb=True,
)

Epoch 1 / 30 | Loss: 0.3153 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 939.99it/s]
Epoch 2 / 30 | Loss: 0.2634 | Precision: 0.9412 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 920.49it/s]
Epoch 3 / 30 | Loss: 0.1750 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 917.15it/s]
Epoch 4 / 30 | Loss: 0.1385 | Precision: 0.9444 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 917.88it/s]
Epoch 5 / 30 | Loss: 0.0893 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 936.23it/s]
Epoch 6 / 30 | Loss: 0.1069 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 916.46it/s]
Epoch 7 / 30 | Loss: 0.0755 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 927.24it/s]
Epoch 8 / 30 | Loss: 0.0616 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 923.78it/s]
Epoch 9 / 30 | Loss: 0.0697 | Precision: 1.0000 | Recall: 1.0000

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval_metrics/accuracy,▁▅▆▆▆▇▇▇▇▇▇▇▇█████████████████
eval_metrics/f1,▁▄▅▅▅▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇████████
eval_metrics/loss,█▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_metrics/precision,▁▃▄▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████
eval_metrics/recall,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_metrics/loss,█▇▅▅▃▃▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▄▁▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁
training_metrics/precision,▅█▁▃▄███████████████████████████████████
training_metrics/recall,██████████████████████▃█▁██████▁████████
eval_metrics/accuracy,99.53775
eval_metrics/f1,0.92105


Epoch 1 / 30 | Loss: 0.3023 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 924.12it/s]
Epoch 2 / 30 | Loss: 0.2581 | Precision: 0.8889 | Recall: 0.9412: 100%|██████████| 771/771 [00:00<00:00, 945.77it/s]
Epoch 3 / 30 | Loss: 0.1133 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 927.72it/s]
Epoch 4 / 30 | Loss: 0.0944 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 910.15it/s]
Epoch 5 / 30 | Loss: 0.0816 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 925.77it/s]
Epoch 6 / 30 | Loss: 0.0414 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 909.82it/s]
Epoch 7 / 30 | Loss: 0.0493 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 936.70it/s]
Epoch 8 / 30 | Loss: 0.0506 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 904.90it/s]
Epoch 9 / 30 | Loss: 0.0456 | Precision: 1.0000 | Recall: 1.0000

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval_metrics/accuracy,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_metrics/f1,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_metrics/loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_metrics/precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_metrics/recall,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/learning_rate,████████████████████████████████████▁▁▁▁
training_metrics/loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_metrics/precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_metrics/recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_metrics/accuracy,99.92296
eval_metrics/f1,0.66667


ValueError: Path is not a file: probe_A_starts.pt

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f32a5b9ce50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f329f317290, execution_count=153 error_before_exec=None error_in_exec=Path is not a file: probe_A_starts.pt info=<ExecutionInfo object at 7f329e295410, raw_cell="# import auto tqdm
import wandb
import torch
impor.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a22566173744149227d/root/github_repositories/alphabetical_probe/joseph.ipynb#X36sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [9]:
model.W_E.device

device(type='cuda', index=0)

In [15]:
import cProfile
import pstats
from io import StringIO
import wandb

import os 

# os.environ["CUDA_LAUNCH_BLOCKING"] = "True"
# os.environ["TOKENIZERS_PARALLELISM"] = "False"

# Wrap your function call in another function for cProfile to hook into
def profile_all_probe_training_runner():
    probe_weights_tensor = all_probe_training_runner(
        embeddings=model.W_E,
        vocab=vocab,
        alphabet="A",
        criteria_mode="starts",
        probe_type="linear",
        num_epochs=1,
        batch_size=32,
        learning_rate=0.005,
        train_test_split=0.9,
        num_workers=16,
        rebalance=True,
        use_wandb=False,
    )


# Create a Profile object
pr = cProfile.Profile()
# Enable the profiler
pr.enable()

# Call the function you wish to profile
profile_all_probe_training_runner()

# Disable the profiler
pr.disable()

# Create a StringIO object to output your profile results
s = StringIO()

# Sort the stats by cumulative time taken and print to stdout
ps = pstats.Stats(pr, stream=s).sort_stats("cumulative")
ps.print_stats()

# Print the result of the profile to the console
print(s.getvalue())

  0%|          | 0/730 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/730 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/alphabetical_probe/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/alphabetical_probe/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/alphabetical_probe/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/opt/conda/envs/alphabetical_probe/lib/python3.11/site-packages/torch/utils/data/dataset.py", line 298, in __getitem__
    return self.dataset[self.indices[idx]]
           ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^

# Analysis

In [222]:
train_loader, test_loader = get_letter_dataset(
    criterion="starts",
    target="A",
    embeddings=model.W_E,
    vocab=vocab,
    batch_size=16,
    rebalance=True,
)

for batch in train_loader:
    print(batch[0][0])
    print(batch[0][1].shape)
    print(batch[1])
    # get proportions of positive and negative labels
    print(sum(batch[1]) / len(batch[1]))
    break

import plotly.express as px
import pandas as pd


def plot_batch_predictions(words, embeddings, probe):
    predictions = probe(embeddings).squeeze()
    predictions = predictions.squeeze()

    # make a dataframe
    df = pd.DataFrame(
        {
            "words": words,
            "predictions": predictions.cpu().detach().numpy(),
            "labels": batch[1].cpu().detach().numpy(),
        }
    )
    # convert labels to bool
    df["labels"] = df["labels"].astype(bool)
    # sort by predictions
    df = df.sort_values(by=["predictions"], ascending=True)

    fig = px.bar(
        df,
        y="words",
        x="predictions",
        color="labels",
        title="Probe weights for letter A",
        labels={
            "x": "Word",
            "y": "Probe Logit",
            "labels": "Starts with A",
        },
        text_auto="0.2f",
        text="words",
        height=800,
        # width=1200,
        template="plotly_white",
        category_orders={"index": df.index[::-1]},
    )
    # update the hover template to make the word much larger
    fig.update_traces(hovertemplate="<b>%{y}</b><br><br>%{x}")

    return fig


reverse_vocab = {v: k for k, v in vocab.items()}
words = [reverse_vocab[i.item()].strip("Ġ") for i in batch[0][0]]
plot_batch_predictions(words, batch[0][1], probe_weights_tensor["A"])

tensor([24021, 31554, 24035, 23773, 29095,  2758,  7731, 42585,  7537, 19408,
         6882, 43406, 49145, 45661, 16784, 25509])
torch.Size([16, 4096])
tensor([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0])
tensor(0.5000)


In [224]:
batch[0][0].device

device(type='cpu')

In [228]:
probe_weights_tensor["A"].fc.weight.data.device

device(type='cuda', index=0)

In [33]:
import wandb

# shut down any sessions
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9fedb359d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f9fedb01050, raw_cell="import wandb

# shut down any sessions
wandb.finis.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a22566173744149227d/root/github_repositories/alphabetical_probe/joseph.ipynb#X50sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9fedb359d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f9fa0535790, execution_count=33 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f9fedb01050, raw_cell="import wandb

# shut down any sessions
wandb.finis.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a22566173744149227d/root/github_repositories/alphabetical_probe/joseph.ipynb#X50sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

# Speeding Up Data Loading

In [69]:
# Filter Step
roman_char_regex = re.compile('^[A-Za-z]+$') # A to Z and a to z are Roman characters

tokenizer = model.tokenizer
vocab = tokenizer.get_vocab()
# filter vocab to only include valid words


def filter_vocab(vocab):
    english_words = set(w.lower() for w in words.words())
    new_vocab = {}
    for word, index in vocab.items():
        clean_word = word.lstrip("Ġ").strip().lower()
        if clean_word in english_words and bool(roman_char_regex.match(clean_word)):
            new_vocab[word] = index
    return new_vocab

new_vocab = filter_vocab(vocab)
print(len(new_vocab))

vocab_tokens = list(new_vocab.keys())
original_indices = list(new_vocab.values())
index_mapping = {v: i for i, (k, v) in enumerate(new_vocab.items())}
print(len(index_mapping))

new_embeddings = model.W_E[list(new_vocab.values())]
print(new_embeddings.shape)


# word = "almost"
# idx = vocab[word]
# torch.testing.assert_close(model.W_E[idx], new_embeddings[index_mapping[idx]])

# Label Steps

regex_pattern = get_regex_pattern("A", "starts")
labels = [bool(re.search(regex_pattern, k.strip().strip("Ġ"))) for k in vocab_tokens]
labels = torch.tensor(labels, dtype=torch.bool)
labels[:10]
print(len(labels))

25950
25950
torch.Size([25950, 4096])
25950


In [70]:
# Data Step 
from torch.utils.data import TensorDataset
dataset = TensorDataset(new_embeddings, labels)

In [71]:
test_proportion = 0.2
batch_size = 32


train_size = int((1 - test_proportion) * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Creating data loaders for the train and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=make_weights_for_balanced_classes,
)
rebalance = True
# rebalance
if rebalance:
    n_total = len(labels)
    n_pos = sum(labels)
    n_neg = len(labels) - n_pos
    weight_per_class = {1: n_total / n_pos, 0: n_total / n_neg}
    weights = [weight_per_class[1]] * n_pos + [weight_per_class[0]] * n_neg
    train_weights = [weights[i] for i in train_dataset.indices]
    sampler = WeightedRandomSampler(train_weights, len(train_weights))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=0)
else:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0)
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

In [121]:
len(dataset.tensors[1])

25950

In [72]:
from torch.utils.data import Dataset
import torch


import time 



dataset = TensorDataset(new_embeddings, labels)
start = time.time()

print(dataset[0])
    
end = time.time() - start
print(end)

(tensor([-3.2390e-02,  6.7799e-03,  1.5451e-02,  ...,  8.8690e-03,
        -1.0720e-02, -7.4208e-05], device='cuda:0', grad_fn=<SelectBackward0>), tensor(False))
0.001961231231689453


In [73]:
len(train_loader)

649

In [ ]:
import tempfile
import os
import torch
import wandb


def create_and_log_artifact(tensor, name, artifact_type, description):
    # Use a temporary file to save the tensor
    with tempfile.NamedTemporaryFile(delete=True, suffix=".pt") as tmp:
        torch.save(tensor, tmp.name)

        # Create a new artifact
        artifact = wandb.Artifact(
            name=name,
            type=artifact_type,
            description=description,
        )
        artifact.add_file(tmp.name)

        # Log the artifact
        wandb.log_artifact(artifact)


hidden_dim = 4096
num_hidden_layers = 2


def get_mlp_weights(model):
    # Extracts weights from a trained MLP model and returns them as a dictionary.
    #
    #:param model: The trained model.
    #:return: A dictionary containing weights and biases for each layer.

    weights = {}
    weights["input_weight"] = model.fc_input.weight.data.clone().detach()
    weights["input_bias"] = model.fc_input.bias.data.clone().detach()

    for idx, layer in enumerate(model.fc_hidden):
        weights[f"hidden_{idx+1}_weight"] = layer.weight.data.clone().detach()
        weights[f"hidden_{idx+1}_bias"] = layer.bias.data.clone().detach()

    weights["output_weight"] = model.fc_output.weight.data.clone().detach()
    weights["output_bias"] = model.fc_output.bias.data.clone().detach()

    return weights